In [1]:
from feature_engineering import generate_features
from get_data import get_raw_data
import pandas as pd
from feature_engineering import get_sp_500_data

In [2]:
data=generate_features(get_raw_data('TSLA'))


[*********************100%%**********************]  1 of 1 completed


In [3]:
data.head(-10)


,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,ratio_std_volume_5_365,ratio_std_volume_30_365,return_1,return_5,return_30,return_365,moving_avg_5,moving_avg_30,moving_avg_365,close
Date,,,,,,,,,,,,,,,,,,,,,
2021-09-10,253.199997,251.136673,251.619995,254.033340,250.543335,42233100.0,248.507330,238.241905,211.371071,1.043088,...,0.144253,0.176977,0.001313,0.028293,0.063198,1.060882,0.032384,0.059679,4.521669,245.423340
2021-09-13,246.736664,253.199997,245.423340,254.203339,244.839996,45552600.0,248.765997,238.693492,211.853783,1.042198,...,0.121335,0.164318,-0.024627,0.005298,0.040194,0.982738,0.031605,0.061001,4.498992,247.666672
2021-09-14,247.523331,246.736664,247.666672,248.259995,236.283340,68857500.0,249.394666,239.022858,212.343571,1.043393,...,0.170012,0.176476,0.009141,0.012855,0.028730,0.993454,0.023757,0.060118,4.477043,248.163330
2021-09-15,248.333328,247.523331,248.163330,251.490005,245.466660,55574700.0,248.832666,239.456508,212.773294,1.039156,...,0.165397,0.178744,0.002005,-0.011196,0.038094,0.774200,0.020246,0.056466,4.454876,251.943329
2021-09-16,250.943329,248.333328,251.943329,252.286667,246.119995,46073100.0,248.963333,240.562222,213.178148,1.034923,...,0.173805,0.171650,0.015232,0.002600,0.101520,0.680518,0.011984,0.053408,4.427961,252.330002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-17,157.639999,156.740005,157.110001,158.190002,153.750000,97000000.0,167.200000,171.545239,221.517579,0.974670,...,0.558953,0.753186,-0.027062,-0.111771,-0.039494,-0.150757,0.014559,-0.085416,0.056078,155.449997
2024-04-18,151.250000,157.639999,155.449997,158.330002,153.779999,82439700.0,163.938000,170.671430,221.392222,0.960547,...,0.563034,0.743554,-0.010566,-0.094958,-0.105581,-0.168894,-0.020097,-0.080953,0.057217,149.929993
2024-04-19,148.970001,151.250000,149.929993,152.199997,148.699997,96098800.0,159.003998,169.652857,221.255793,0.937231,...,0.571020,0.732984,-0.035510,-0.141294,-0.124854,-0.186534,-0.043104,-0.079649,0.058337,147.050003


In [4]:
data.shape

(668, 38)

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Sequential
from keras.layers import  LSTM, Dense,Dropout


In [6]:
# here we are seperating the data 
trainning_set = data[:'2024-02-01'].iloc[:,1:2].values
test_set = data['2024-02-01':].iloc[:,1:2].values

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = scaler.fit_transform(trainning_set)


X_train = []
Y_train = []


for i in range(1, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-1:i, 0])
    Y_train.append(training_set_scaled[i, 0])

# Convert lists to NumPy arrays
X_train, y_train = np.array(X_train), np.array(Y_train)

# Reshape X_train for LSTM input shape (batch_size, timesteps, input_dim)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [7]:
X_train.shape

(601, 1, 1)

In [8]:

# pre-processing the data 
dataset_total = pd.concat((data["close"][:'2024-02-01'],data["close"]['2024-02-01':]),axis=0)
inputs = dataset_total[len(dataset_total)-len(test_set) - 500:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

# making the test data
X_test = []
for i in range(500,len(inputs)):
  X_test.append(inputs[i-1:i,0])

X_test_no_shape = np.array(X_test)
X_test = np.reshape(X_test_no_shape, (X_test_no_shape.shape[0],X_test_no_shape.shape[1],1))


test_set=scaler.transform(test_set)

In [9]:
print(test_set)

[[0.27231173]
 [0.27717444]
 [0.26595777]
 [0.26342916]
 [0.24057447]
 [0.27613704]
 [0.27879534]
 [0.28262065]
 [0.28887736]
 [0.26255391]
 [0.26680067]
 [0.27931404]
 [0.32113333]
 [0.30190944]
 [0.29292962]
 [0.29500438]
 [0.29925113]
 [0.28946086]
 [0.32755209]
 [0.31581677]
 [0.32800594]
 [0.31614097]
 [0.31033811]
 [0.25950661]
 [0.24958669]
 [0.2313029 ]
 [0.2544818 ]
 [0.23486886]
 [0.24238987]
 [0.22708854]
 [0.20997181]
 [0.19502708]
 [0.21726587]
 [0.22485169]
 [0.22692644]
 [0.23791616]
 [0.20647065]
 [0.21318117]
 [0.24501573]
 [0.25419005]
 [0.24135247]
 [0.23720296]
 [0.20018154]
 [0.19781504]
 [0.21742797]
 [0.21421857]
 [0.21506142]
 [0.22663469]
 [0.22705609]
 [0.22546764]
 [0.22478684]
 [0.21797907]
 [0.17421469]
 [0.1771323 ]
 [0.15641716]
 [0.14902584]
 [0.12176224]
 [0.13074205]
 [0.19398968]
 [0.1814115 ]
 [0.21347297]
 [0.27691509]
 [0.27224688]
 [0.2561027 ]
 [0.25889065]
 [0.2564269 ]
 [0.26193796]]


In [10]:
from tensorboard.plugins.hparams import api as hp

In [11]:
def mae(y_predicted, y_true):
    total_error = 0
    for yp, yt in zip(y_predicted, y_true):
        total_error += abs(yp - yt)
    
    mae = total_error/len(y_predicted)
    
    return mae

In [12]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([64, 128]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([64, 128]))
HP_NUM_UNITS_3 = hp.HParam('num_units_3', hp.Discrete([32, 64]))
HP_NUM_UNITS_4 = hp.HParam('num_units_4', hp.Discrete([32, 64]))
HP_NUM_UNITS_5 = hp.HParam('num_units_5', hp.Discrete([16, 32]))
HP_DROPOUT_1 = hp.HParam('dropout_1', hp.RealInterval(0.1,0.2))
HP_DROPOUT_2 = hp.HParam('dropout_2', hp.RealInterval(0.1, 0.2))


METRIC_MAE = 'mean_absolute_error'


with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_1, HP_NUM_UNITS_2, HP_NUM_UNITS_3, HP_NUM_UNITS_4, HP_NUM_UNITS_5, HP_DROPOUT_1, HP_DROPOUT_2],
     metrics=[hp.Metric(METRIC_MAE, display_name='mean_absolute_error')]
    )  


In [13]:
def train_test_model(hparams):
  
  model = Sequential()

  model.add(LSTM(units=hparams[HP_NUM_UNITS_1], return_sequences=True, input_shape=(X_train.shape[1], 1)))
  model.add(Dropout(hparams[HP_DROPOUT_1]))

  model.add(LSTM(units=hparams[HP_NUM_UNITS_2], return_sequences=True))
  model.add(Dropout(hparams[HP_DROPOUT_2]))

  model.add(LSTM(units=hparams[HP_NUM_UNITS_3]))
  model.add(Dropout(0.1))

  model.add(Dense(units=hparams[HP_NUM_UNITS_4]))
  model.add(Dropout(0.1))

  model.add(Dense(units=hparams[HP_NUM_UNITS_5]))
  model.add(Dropout(0.1))

  model.add(Dense(units=1))
                         
  model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='mean_squared_error', metrics=['mae'])
  
  callback = keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=10,
    restore_best_weights=True)

  model.fit(X_train, y_train, epochs=25,batch_size=128,callbacks=[callback]) # Run with 1 epoch to speed things up for demo purposes
  loss = model.evaluate(X_test, test_set)[0]  
  
  return loss
  
  

In [14]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams) 
        loss = train_test_model(hparams)  
        tf.summary.scalar('mean_absolute_error', loss, step=1)
       


rm ./logs/  ------ clean logs


In [15]:
session_num = 0

for num_units_1 in HP_NUM_UNITS_1.domain.values:
    for num_units_2 in HP_NUM_UNITS_2.domain.values:
        for num_units_3 in HP_NUM_UNITS_3.domain.values:
            for num_units_4 in HP_NUM_UNITS_4.domain.values:
                for num_units_5 in HP_NUM_UNITS_5.domain.values:
                    for dropout_rate_1 in (HP_DROPOUT_1.domain.min_value, HP_DROPOUT_1.domain.max_value):
                        for dropout_rate_2 in (HP_DROPOUT_2.domain.min_value, HP_DROPOUT_2.domain.max_value):
                                    hparams = {
                                        HP_NUM_UNITS_1: num_units_1,
                                        HP_NUM_UNITS_2: num_units_2,
                                        HP_NUM_UNITS_3: num_units_3,
                                        HP_NUM_UNITS_4: num_units_4,
                                        HP_NUM_UNITS_5: num_units_5,
                                        HP_DROPOUT_1: dropout_rate_1,
                                        HP_DROPOUT_2: dropout_rate_2,
                                    }
                                    run_name = "run-%d" % session_num
                                    print('--- Starting trial: %s' % run_name)
                                    print({h.name: hparams[h] for h in hparams})
                                    run('logs/hparam_tuning3/' + run_name, hparams)
                                    session_num += 1



--- Starting trial: run-0
{'num_units_1': 64, 'num_units_2': 64, 'num_units_3': 32, 'num_units_4': 32, 'num_units_5': 16, 'dropout_1': 0.1, 'dropout_2': 0.1}

Epoch 1/25


5/5 [==============================] - 4s 8ms/step - loss: 0.2374 - mae: 0.4457
Epoch 2/25
5/5 [==============================] - 0s 5ms/step - loss: 0.1638 - mae: 0.3571
Epoch 3/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0956 - mae: 0.2577
Epoch 4/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0480 - mae: 0.1730
Epoch 5/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0401 - mae: 0.1612
Epoch 6/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0469 - mae: 0.1796
Epoch 7/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0393 - mae: 0.1598
Epoch 8/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0355 - mae: 0.1493
Epoch 9/25
5/5 [==============================] - 0s 5ms/step - loss: 0.0362 - mae: 0.1497
Epoch 10/25
5/5 [===